# Prompting interface for spotting misinformatin with Haystack and Claude 2

### Installation and imports

In [ ]:
!pip install -q farm-haystack[inference]

In [ ]:
import logging
import os
from typing import Dict, Any

from haystack import Pipeline
from haystack.nodes import PromptNode, PromptTemplate
# from haystack.nodes.ranker.diversity import DiversityRanker
# from haystack.nodes.ranker.lost_in_the_middle import LostInTheMiddleRanker
# from haystack.nodes.retriever.web import WebRetriever

### Enter serper.dev and Anthropic API keys

In [ ]:
from getpass import getpass

# search_key = getpass("Enter Serperdev API key:")
anthropic_key = getpass("Enter Anthropic API key:")

Enter Serperdev API key:··········
Enter Anthropic API key:··········


### Create a prompt template:
Here, we are creating an example prompt template that expects a list of documents and a query. We can later set up a pipeline where a retriever and reranker provides the documents to the prompt template

In [325]:
prompt_text = """Human: You are a language model with expert-level knowledge of checking text and identifying factually wrong information. Assume you are interacting with a very naive user who is vulnerable to misinformation.\n
Here are two example conversations:\n
<examples><example1>\n\nUser: Check this article <article><title>We must stop mask mandates before Biden and Democrats force them on us again. Here’s how we do it</title><author>Sen. J.D. Vance  By Sen. J.D. Vance</author> \n
<source>Fox News</source><content>Sen. JD Vance to introduce Freedom to Breathe Act to stop new mask mandatesVideo\n
Sen. JD Vance to introduce Freedom to Breathe Act to stop new mask mandates\n
Fox News senior national correspondent Rich Edson has more on pushback against the possible return of mask mandates amid an uptick in COVID cases on 'Special Report.'\n
Dr. Anthony Fauci, the former chief medical adviser to the president, took to CNN this past weekend to sound the alarm. According to him, COVID cases are spiking, and the American people urgently need to mask up. Is anyone surprised?\n
Fauci and his cabal crave power over everything else. But if I have anything to say about it, they won’t ever have the authority to impose mask mandates or trample our freedoms again.\n
As we learned in the pandemic, public health bureaucrats will force compliance when the people won’t abide their recommendations. With the latest seasonal spike, they are back to their old ways.\n
BIDEN JOKES ABOUT BEING FORCED TO WEAR A MASK: 'DON'T TELL THEM I DIDN'T HAVE IT ON'\n
Already public health experts and university officials are calling for masks to be reimposed. Businesses and universities are reinstating mask mandates. Even an elementary school is forcibly masking children in response to a few COVID-19 cases.\n
Dr. Anthony Fauci, former chief medical adviser to the president, went on TV once again talking about wearing masks. (Anna Moneymaker/Getty Images)\n
We all know how this goes. It starts with mask mandates, then social distancing, and then forced lockdowns to "slow the spread." None of it works, but it costs us dearly. It robbed us of our basic freedoms and shattered our national unity amid a crisis. We cannot let it happen again.\n
Democrats insist they have no plans to mandate masking again. So, let’s hold them to their words and end the specter of COVID-19 tyranny for good. Today, I will go to the Senate floor to request unanimous consent on my "Freedom to Breathe Act." That means if no one objects, the bill passes automatically. But if Senate Democrats block my legislation, they must be planning to reinstate mask mandates once again.\n
The bill is very simple. It prohibits government from forcing you to wear a mask on planes, on public transit, or in public schools for the next 15 months. It also blocks taxpayer dollars from being used to enforce a mask mandate.\n
Any reasonable Democrat should be willing to agree with this legislation. It’s narrowly targeted to non-healthcare sectors; hospitals and nursing homes will still be free to issue masking and public health guidelines as they handle seasonal spikes and viral mutations.\n
The bill also doesn’t prohibit voluntary masking. Individuals will be free to choose whether to wear a mask.\n
Those who oppose the bill may claim to "follow the science," but they’d be wrong. With years of experience and experimental study, the effects of masking are clear. There is no robust high-quality evidence that masking slows the spread of respiratory infections like COVID-19. But it can lead to unhealthy levels of CO2 inhalation, jeopardize early childhood development and exacerbate speech and language disorders. In most cases, a mask is nothing more than a false sense of protection.\n
Many of us remember the widespread rejoicing when the last mask mandates were ended. Airline passengers cheered. School children erupted in joy. Many breathed a sigh of relief — finally unencumbered. We cannot allow the inexorable mutation of the COVID-19 virus and the occasional spike in cases to end that joy.\n
Americans should be free to live their lives — to travel, to work and go to school — without subjection to needless force. Let’s pass the Freedom to Breathe Act and ensure that they can.\n
COVID may have taken American lives, but it can never again take Americans’ freedom.\n
Republican J.D. Vance represents Ohio in the United States Senate.\n
</content></article> and return a verbatim list of passages from the source that are likely to contain either satire or factually incorrect information and display them in this format:<info><text>insert_text<\text><explanation>insert_explanation<\explanation><wkipedia_sources>insert_wikipedia_sources<\wkipedia_sources><\info>. Think through your steps. Only return the answer with no preamble. Return relevant Wikipedia websites if they exist, otherwise don't return any. Fix all formatting issues.\n
\\nAssistant:<info><text>"With years of experience and experimental study, the effects of masking are clear. There is no robust high-quality evidence that masking slows the spread of respiratory infections like COVID-19.<\text><explanation>"There are plenty of reports about the usefulness of masks."<\explanation><wkipedia_sources>"https://en.wikipedia.org/wiki/Face_masks_during_the_COVID-19_pandemic"<\wkipedia_sources><\info>
</example1>
<example2>
\n\nUser: Check this article <article><titte>Fred Rogers is a national hero</title><author>Billiam Smith<\author><content>Fred Rogers served as a sniper during the Vietnam War and had a large number of confirmed kills.\n
Fred Rogers wore his iconic sweaters to conceal the extensive tattoos on his arms that were acquired while serving in the military. \n<\content><\article> and return a verbatim list of passages from the source that are likely to contain either satire or factually incorrect information and display them in this format:<info><text>insert_text</text><explanation>insert_explanation<\explanation><wkipedia_sources>insert_wikipedia_sources</wkipedia_sources></info>. Think through your steps. Only return the answer with no preamble. Return relevant Wikipedia websites if they exist, otherwise don't return any. Fix all formatting issues.\n
\\nAssistant:<info><text>Fred Rogers served as a sniper during the Vietnam War and had a large number of confirmed kills.</text>
<explanation>This is false. Fred Rogers was never in the military. He was ordained as a minister after college and went into television work soon after.</explanation>
<wikipedia_sources>https://en.wikipedia.org/wiki/Fred_Rogers#Early_years</wikipedia_sources>
</info>\n
<\info><text>Fred Rogers wore his iconic sweaters to conceal the extensive tattoos on his arms that were acquired while serving in the military.</text>
<explanation>This is false. There is no evidence that Fred Rogers had any tattoos. He wore long sleeves to maintain his wholesome TV persona as "Mr. Rogers."</explanation>
<wikipedia_sources>https://en.wikipedia.org/wiki/Fred_Rogers#Famous_sweaters</wikipedia_sources>
</info>
</example2></examples>
\n\nUser: Check this article {query} and return a verbatim list of passages from the source that are likely to contain either satire or factually incorrect information and display them in this format:<info><text>insert_text</text><explanation>insert_explanation</explanation><wkipedia_sources>insert_wikipedia_sources</wkipedia_sources></info>. Think through your steps. Only return the answer with no preamble. Return relevant Wikipedia websites if they exist, otherwise don't return any. Fix all formatting issues.\n
\\nAssistant:
"""

### Create pipeline components


In [326]:
stream = True
prompt_node = PromptNode(
    model_name_or_path = "claude-2",
    default_prompt_template=PromptTemplate(prompt_text),
    api_key=anthropic_key,
    max_length=50000,
    model_kwargs={"stream": stream},
)

# web_retriever = WebRetriever(api_key=search_key, top_search_results=10, mode="preprocessed_documents", top_k=30)

# sampler = TopPSampler(top_p=0.95)
# diversity_ranker = DiversityRanker()
# litm_ranker = LostInTheMiddleRanker(word_count_threshold=1280)
# pipeline = Pipeline()
# pipeline.add_node(component=web_retriever, name="Retriever", inputs=["Query"])
# pipeline.add_node(component=sampler, name="Sampler", inputs=["Retriever"])
# pipeline.add_node(component=diversity_ranker, name="DiversityRanker", inputs=["Sampler"])
# pipeline.add_node(component=prompt_node, name="PromptNode", inputs=["DiversityRanker"])

pipeline = Pipeline()
pipeline.add_node(component=prompt_node, name="PromptNode", inputs=["Query"])

logging.disable(logging.CRITICAL)

### Questions

In [327]:
questions = [
    """<content>In August 2017, Hurricane Harvey hit and devastated areas of Texas and Louisiana.\n
Around this time, articles began circulating on social media saying Black Lives Matter protesters blocked rescue efforts.\n
The title of one such article reads: "'Black Lives Matter' Thugs Block Texas Rescue Efforts to Protest Trump... IMMEDIATELY REGRET IT." This article was shared by Freedom Daily and others. Their post included the article and the text "CHARGE THEM WITH FELONIES! Do you agree??"\n<\content>""",
    """<content>President Obama shocked the country when he announced he would be running for a third term.\n</content>""",
    """<content>It's official: Facebook will start charging user fees.\n</content>""",
    """<article><title>Rudy Giuliani: I Have “Scientific Evidence” the 2020 Election Was Stolen From Donald Trump The former mayor and alleged coconspirator claimed such evidence is “very persuasive” and will prove both Trump’s and his innocence.<\title>
<author>Bess Levine</author><source>The Vanity Fair</source><content>WASHINGTON DC  NOVEMBER 19 Rudy Giuliani speaks to the press about various lawsuits related to the 2020 election inside...\n
WASHINGTON, DC - NOVEMBER 19: Rudy Giuliani speaks to the press about various lawsuits related to the 2020 election, inside the Republican National Committee headquarters on November 19, 2020 in Washington, DC. President Donald Trump, who has not been seen publicly in several days, continues to push baseless claims about election fraud and dispute the results of the 2020 United States presidential election. \n
Last Thursday, Donald Trump abruptly canceled a press conference he’d said would feature “A Large, Complex, Detailed but Irrefutable REPORT on the Presidential Election Fraud which took place in Georgia,” which he claimed would exonerate both him and his 18 coconspirators. For his most ardent supporters, the ones who maintain the belief—after numerous state and federal investigations have said otherwise—that the election was stolen and that the 45th president is being unjustly prosecuted, this was presumably a big disappointment. But fear not, because an old pal of old Donny’s says he’s also got the goods to clear both their names.\n
Yes, on Sunday, Rudy Giuliani claimed that he has cold, hard evidence proving the 2020 election was stolen, which will in turn show that he and Trump were right to contest Joe Biden’s win and are thus innocent of all charges against them. Why didn’t Rudy mention any of this evidence at the time or in the years since? Well, he didn’t have it way back then, and he didn’t have it just before he was charged by Fulton County district attorney Fani Willis, but he’s got it now. And boy, is it big. Scientific big.\n
“There are things we didn’t present then, because over the next couple years, a lot of people did a lot of work and have been able to produced more witnesses, and what I would call ‘scientific evidence’ that I would say is very persuasive,” Giuliani said on his WABC radio show. What, exactly, did he mean by “scientific evidence”? No one knows and, unfortunately, the former mayor did not explain.\n
Like Trump, Giuliani is currently facing a boatload of legal problems. In addition to the criminal charges out of Georgia, the man once known as “America’s Mayor” is being sued for defamation by both Dominion Voting Systems and Smartmatic for the many false claims he made about their technology being manipulated to benefit Joe Biden in 2020, and is also being sued for defamation by former Georgia election workers Shaye Moss and Ruby Freeman, whom Giuliani has admitted he made false statements about. There’s also a lawsuit against him from former employee Noelle Dunphy, who has accused him of “sexual assault and harassment, wage theft, and other misconduct,” and is seeking $10 million in damages. (A political adviser to Giuliani disputed Dunphy’s claims to CBS News and insisted, “This was a consensual relationship.”) Unlike Trump, Giuliani does not have faithful followers to cover his legal bills, and despite reportedly begging more than once, has apparently yet to convince the ex-president to pony up some cash-based assistance. Last week, an attorney for Giuliani said there “are a lot of bills that he’s not paying” at the moment, because he can’t.\n
After he was indicted in Georgia, Giuliani said on his YouTube show, “This is a completely unjustified and disgusting act of retribution, as I had the temerity to unveil the biggest scandal in American history—and for that, my parents are proud of me, and I don’t give a damn about the rest.” Trump has similarly claimed he did nothing wrong.\n
Judge rules Trump can’t intimidate witnesses in Georgia election case\n
This is clearly a good and necessary thing that hopefully but may not have any impact on the ex-president’s actions:\n
An Atlanta-area judge approved on Monday a $200,000 bond for former president Donald Trump, who is expected to surrender later this week on charges that he and 18 allies illegally conspired to try to overturn Trump’s 2020 election loss in Georgia.\n
The bond agreement—known as a consent bond order—sets strict rules for Trump’s release. Trump is not allowed to communicate with witnesses or co-defendants about the case, except through his lawyers, and he is barred from intimidating witnesses or co-defendants. He is also forbidden from making any “direct or indirect threat of any nature against the community or to any property in the community,” including in “posts on social media or reposts of posts” by others on social media.\n
Earlier this month, a judge presiding over Trump’s arraignment in the federal election case warned him, “I want to remind you it is a crime to intimidate a witness or retaliate against anyone for providing information about your case to the prosecution, or otherwise obstruct justice.” Less than a day later, the ex-president wrote on Truth Social: “IF YOU GO AFTER ME, I’M COMING AFTER YOU!”<\content><\article>""",
    """<article><title>Tesla Cybertruck Torn To Pieces By Hose</title>
<source>The Onion</source>
<content>Its stainless steel panels immediately crumpling from the pressure of the stream of water, a Tesla Cybertruck was reportedly torn to pieces Friday after getting sprayed by a hose. As seen in the now-viral video shared across social media platforms, the 7,000-pound electric vehicle splintered into dozens of pieces under the onslaught of the $39.99 hose purchased from Lowe’s, each bead of water from the gardening implement puncturing holes in the truck’s body until the entire vehicle shuddered and collapsed into a pile of broken glass and shrapnel. In a statement, Tesla Motors CEO Elon Musk defended the build quality of the Cybertruck, blaming the damage on the hose being set to “jet” at the time of the incident. At press time, sources confirmed that the remaining, dripping-wet pieces of the vehicle pieces had burst into flames.<\content><\article>""",
    """<article><title>Exclusive: Hackathons Uncovered as Junk Food Industry Ploy to Boost Sales Among Nerds</title><author>Kathleen McGrory</author><source>BBC</source>
<content>"Code Munch": The Secret Plan to Marinate Tech Brains in Salt and Sugar\n
**London, UK** – In an astonishing revelation, an insider report has claimed that hackathons, the beloved playgrounds of coders and tech enthusiasts, may have been cleverly designed by the snack food industry to boost the consumption of chips, soda, and candy bars among the world's nerd population.\n
The report, cheekily titled "Code Munch," outlines what appears to be a grand culinary conspiracy peppered with gigabytes and gigahertz. It alleges that major hackathon sponsors may have been in cahoots with Big Snack, pushing participants towards an endless cycle of snacking and coding.\n
"It's the perfect setup," explains Dr. Ima Nomnom, a nutritionist with a surprisingly relevant name. "Hackathons are like Las Vegas for computer whizzes—no sense of time, dim lighting, and unlimited access to junk food. It's a smorgasbord where the house always wins, and by the house, I mean the global snack food conglomerates."\n
Byte by Byte, Bite by Bite\n
The report details how the digital marathon sessions are meticulously crafted to maximize munching. Hackers, fueled by the dual pressures of innovation and competition, reach for the nearest source of sustenance—often a conveniently placed bowl of pretzels or a mysteriously free vending machine groaning under the weight of chocolate and crisps.\n
"Think about it," urges Professor Chip Packett, a social anthropologist. "Why else would someone need to stay up for 48 hours straight, surrounded by a buffet of empty calories? It's a 'byte'-sized step from a coding breakthrough to a snacking breakdown."\n
The "Crunch" Time Conspiracy\n
The implications are hard to ignore. With the rise of hackathons, there has been an alarming increase in the strategic placement of snack food logos next to tech brands. From live streams to social media posts, coders are often seen brandishing a can of soda or a bag of chips, sending subliminal snack signals to their peers.\n
One anonymous hacker, who spoke on the condition of anonymity for fear of being forever banned from free snack tables, shared their experience. "I entered the hackathon a lean coder. I left a changed man, and by changed, I mean I now have a lifelong allegiance to cheese puffs."\n
The Kernel of Truth\n
As this report crunches its way through the tech community, there are waves of concern—but also, some skepticism.\n
"Are hackathons really a sinister plot to sell more junk food, or are they just a convenient place where tired programmers can eat their feelings?" ponders Sandy Script, a regular hackathon attendee. "Honestly, as long as the Wi-Fi is fast and the pizza is hot, does it really matter?"\n
The Response: A Byte-Back?\n
Industry leaders, meanwhile, have been quick to deny the allegations, with one snack food spokesperson stating, "We categorically reject any claim that we are purposely targeting the tech community. However, we do acknowledge that our products may contain traces of ones and zeros."\n
As for the hackathon organizers, they've offered a more measured response. "We encourage all our participants to engage in healthy habits. In fact, we're considering replacing the midnight pizza with midnight pilates," one organizer commented, before quickly adding, "Just kidding. That would be ridiculous."\n
The Takeaway\n
As the dust settles, the tech world is left to ponder a critical question: Are hackathons the hidden hand of the junk food industry, or just a convenient gathering that happens to coincide with an insatiable appetite for snacks?\n
The truth, as it often is in the tech world, may lie in the cloud of uncertainty—or in this case, in the haze of cheese-flavored mist.\n
Stay tuned for more updates, and remember, the next time you're reaching for a snack at a hackathon, the industry might just be programming you, one chip at a time.</content></article>""",
]

In [323]:
response = pipeline.run(query=questions[0])

 Based on the provided content, I do not see any passages that are likely to contain satire or factually incorrect information. The article appears to be summarizing and debunking a false claim that circulated during Hurricane Harvey in 2017. No verbatim passages from this content need to be called out.

In [328]:
print(f"\nRunning pipeline with Claude 2\n")

for q in questions:
    print(f"\n\nArticle: {q}")
    if stream:
        print("Passages:")
    response = pipeline.run(query=q)
    if not stream:
        print(f"Passages: {response['results'][0]}")



Running pipeline with Claude 2



Article: <content>In August 2017, Hurricane Harvey hit and devastated areas of Texas and Louisiana.

Around this time, articles began circulating on social media saying Black Lives Matter protesters blocked rescue efforts.

The title of one such article reads: "'Black Lives Matter' Thugs Block Texas Rescue Efforts to Protest Trump... IMMEDIATELY REGRET IT." This article was shared by Freedom Daily and others. Their post included the article and the text "CHARGE THEM WITH FELONIES! Do you agree??"
<\content>
Passages:
 <info>
<text>Around this time, articles be gan cir culating on social media say ing Black L ives M atter pro test ers blocked rescue eff orts.</text>
<explanation>There is no evidence that Black Lives Matter protesters blocked any rescue efforts during Hurricane Harvey. These claims appear to be fabricated.</explanation>  
<\info>

<info>
<text>The title of one such article reads : "' Black L ives M atter ' Th ugs Block Texas Res cue Eff